In [1]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
from ipywidgets import IntProgress
from IPython.display import display

def log_progress(sequence, every=10):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)

    for index, record in enumerate(sequence):
        
        if index % every == 0:
            progress.value = index
        yield record
    progress.value = len(sequence)

### Space

In [3]:
mesh_dx = 20
mesh = fe.RectangleMesh(fe.Point(-2, -2), fe.Point(2, 2), mesh_dx, mesh_dx)
space_v = fe.VectorFunctionSpace(mesh, 'P', 2)
space_p = fe.FunctionSpace(mesh, 'P', 1)

v = fe.TrialFunction(space_v)
test_v = fe.TestFunction(space_v)
v0=fe.Constant((0,0))

p = fe.TrialFunction(space_p)
test_p = fe.TestFunction(space_p)
p0=fe.Constant(0)


### Boundary

In [4]:
# Flows in the box
walls = 'near(x[1],0) || near(x[1],1)'
# def walls(x, on_bound):
#     return fe.near(x[1],0) or fe.near(x[1],1)
left = 'near(x[0],0)'
right = 'near(x[0],1)'

bcu_wall = fe.DirichletBC(space_v, fe.Constant((0,0)), walls)
bcp_left=fe.DirichletBC(space_p,fe.Constant(0),left)
bcp_right=fe.DirichletBC(space_p,fe.Constant(0),right)

bcu=[bcu_wall]
bcp=[bcp_left,bcp_right]

### Operators

In [5]:
Grad= fe.nabla_grad
Dot=fe.dot
dx=fe.dx
ds=fe.ds

### Equations

In [6]:
def Epsilon(f):
    return fe.sym(Grad(f))


def sigma(f):
    return 2 * Epsilon(f) - p * fe.Identity(len(f))

n=1
L = 2 * Dot((v - v0) / 2, test_v) * dx + Dot(v0, Grad(v0), test_v) * dx +\
    fe.inner(sigma(0.5 * (v + v0)), Epsilon(test_v)) * dx +\
    Dot(p0 * n, test_v) * ds -\
    Dot(2 * Grad(v) * n, test_v) * ds


Cannot determine geometric dimension from expression.


UFLException: Cannot determine geometric dimension from expression.